### 다음에 시도할 내용
- KNN으로 도전

- train dataset으로 어떤 feature 조합이 제일 정확도가 높은지 알아보기

- RandomForestRegression으로 도전

- 아무래도 정규화를 0~1구간으로 해야겠다.

- Logistic Regression

In [37]:
import pandas as pd

train = pd.read_csv('C:/Users/user/LikeLion_AI/DACON/Parking_demand/parking_data_preprocessing_train_07_17.csv')
test = pd.read_csv('C:/Users/user/LikeLion_AI/DACON/Parking_demand/parking_data_preprocessing_test_07_17.csv')
sub = pd.read_csv('C:/Users/user/LikeLion_AI/DACON/Parking_demand/CSV/sample_submission.csv')
age = pd.read_csv('C:/Users/user/LikeLion_AI/DACON/Parking_demand/CSV/age_gender_info.csv')

train.shape, test.shape, sub.shape, age.shape

((2869, 16), (1007, 16), (150, 2), (16, 23))

In [38]:
train.head()

,index,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,8,C2515,4.083399,1.0,2,5,0.730930,2.950644,6.181818,0,0.158291,0.074998,0.0,3.0,6.845938,205.0
1,9,C2515,4.083399,1.0,2,5,0.945373,0.633047,6.181818,0,0.232098,0.102366,0.0,3.0,6.845938,205.0
2,10,C2515,4.083399,1.0,2,5,0.945373,0.203863,6.181818,0,0.232098,0.102366,0.0,3.0,6.845938,205.0
3,11,C2515,4.083399,1.0,2,5,1.201163,0.396996,6.181818,0,0.355131,0.150595,0.0,3.0,6.845938,205.0
4,12,C2515,4.083399,1.0,2,5,1.201163,0.193133,6.181818,0,0.355131,0.150595,0.0,3.0,6.845938,205.0


In [25]:
train['임대건물구분'] = train['임대건물구분'].astype('category')
train['지역'] = train['지역'].astype('category')
train['공급유형'] = train['공급유형'].astype('category')
train['자격유형'] = train['자격유형'].astype('category')

In [24]:
test.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,지하철,버스정류장,단지내주차면수,분양완료,건물_아파트,건물_상가
0,C1072,0.286389,1,1,5,0.068966,0.061695,0.254545,7,0.017576,0.019613,0.0,0.090909,0.37535,0.0,1.0,0.0
1,C1072,0.286389,1,1,5,0.103448,0.015558,0.254545,0,0.028868,0.026417,0.0,0.090909,0.37535,0.0,1.0,0.0
2,C1072,0.286389,1,1,5,0.103448,0.059549,0.254545,7,0.028868,0.026417,0.0,0.090909,0.37535,0.0,1.0,0.0
3,C1072,0.286389,1,1,5,0.103448,0.063841,0.254545,7,0.028868,0.026417,0.0,0.090909,0.37535,0.0,1.0,0.0
4,C1072,0.286389,1,1,5,0.137931,0.031652,0.254545,7,0.035231,0.031723,0.0,0.090909,0.37535,0.0,1.0,0.0


In [26]:
test['임대건물구분'] = test['임대건물구분'].astype('category')
test['지역'] = test['지역'].astype('category')
test['공급유형'] = test['공급유형'].astype('category')
test['자격유형'] = test['자격유형'].astype('category')

In [12]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso

sel = ['총세대수', '건물_아파트', '건물_상가', '지역', '전용면적', '전용면적별세대수', 
       '자격유형', '임대보증금', '임대료', '지하철', '버스정류장', '단지내주차면수', '분양완료']

X_train = train[sel]
X_test = test[sel]
y_train = train['등록차량수']

ex_X = PolynomialFeatures(degree=3, include_bias=False).fit_transform(X_train)
ex_test_X = PolynomialFeatures(degree=3, include_bias=False).fit_transform(X_test)

L1 = Lasso(alpha=1.2)
L1.fit(ex_X, y_train)

pred = L1.predict(ex_test_X)
pred

C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 47012439.07313462, tolerance: 53144.13773091669
  model = cd_fast.enet_coordinate_descent(


array([551.14943742, 664.16328617, 561.09817578, ..., 285.70626177,
       228.96368416, 315.4718259 ])

In [14]:
import numpy as np

test['등록차량수'] = pred

test['코드별차량수평균'] = test.groupby('단지코드')['등록차량수'].transform(np.mean)

test.head(20)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,분양완료,건물_아파트,건물_상가,등록차량수,코드별차량수평균
0,C1072,0.286389,1,1,5,0.068966,0.061695,0.254545,7,0.017576,0.019613,0.0,0.090909,0.37535,0.0,1.0,0.0,551.149437,578.278480
1,C1072,0.286389,1,1,5,0.103448,0.015558,0.254545,0,0.028868,0.026417,0.0,0.090909,0.37535,0.0,1.0,0.0,664.163286,578.278480
2,C1072,0.286389,1,1,5,0.103448,0.059549,0.254545,7,0.028868,0.026417,0.0,0.090909,0.37535,0.0,1.0,0.0,561.098176,578.278480
3,C1072,0.286389,1,1,5,0.103448,0.063841,0.254545,7,0.028868,0.026417,0.0,0.090909,0.37535,0.0,1.0,0.0,560.490771,578.278480
4,C1072,0.286389,1,1,5,0.137931,0.031652,0.254545,7,0.035231,0.031723,0.0,0.090909,0.37535,0.0,1.0,0.0,574.913013,578.278480
5,C1072,0.286389,1,1,5,0.137931,0.026824,0.254545,7,0.035231,0.031723,0.0,0.090909,0.37535,0.0,1.0,0.0,575.596344,578.278480
6,C1072,0.286389,1,1,5,0.137931,0.105687,0.254545,7,0.035231,0.031723,0.0,0.090909,0.37535,0.0,1.0,0.0,564.435280,578.278480
7,C1072,0.286389,1,1,5,0.137931,0.035408,0.254545,7,0.035231,0.031723,0.0,0.090909,0.37535,0.0,1.0,0.0,574.381534,578.278480
8,C1128,0.522423,1,1,5,0.068966,0.196888,0.163636,7,0.017576,0.019613,0.0,0.181818,0.67395,0.0,1.0,0.0,1013.278424,1068.894391
9,C1128,0.522423,1,1,5,0.068966,0.015558,0.163636,7,0.017576,0.019613,0.0,0.181818,0.67395,0.0,1.0,0.0,1038.941278,1068.894391


In [15]:
test_new = test.drop_duplicates(['단지코드'], keep='first').reset_index()

sub_df = test_new[['단지코드', '코드별차량수평균']]
sub_df.columns = ['code', 'num']

sub_df

,code,num
0,C1072,578.278480
1,C1128,1068.894391
2,C1456,531.527597
3,C1840,516.430928
4,C1332,924.483337
...,...,...
145,C2456,309.008030
146,C1266,321.655709
147,C2152,101.248636
148,C1267,235.458711


In [16]:
sub_df.to_csv('07_18_first_submit.csv', index=False)

import os

os.listdir(os.getcwd())

['.ipynb_checkpoints',
 '07_17_first_submit.csv',
 '07_17_second_submit.csv',
 '07_17_third_submit.csv',
 '07_18_first_submit.csv',
 'CSV',
 'parking_data_preprocessing_test_07_17.csv',
 'parking_data_preprocessing_test_07_18.csv',
 'parking_data_preprocessing_train_07_17.csv',
 'parking_data_preprocessing_train_07_18.csv',
 'Parking_demand_07_12(1).ipynb',
 'Parking_demand_07_12(2).ipynb',
 'Parking_demand_07_14.ipynb',
 'Parking_demand_07_16.ipynb',
 'Parking_demand_07_17.ipynb',
 'Parking_demand_07_18.ipynb',
 'Parking_demand_data_error.ipynb',
 'second_rf_0712.csv']

### 1차 제출 : 133.8081420467

In [39]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

sel = ['총세대수', '지역', '임대건물구분', '공가수', '임대보증금', '임대료', '단지내주차면수']

X = train[sel]
y = train['등록차량수']
test_X = test[sel]

for size in range(10, 41):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size/100)
    print(f'test_size = {size/100}')
    
    for n in range(10, 11):
        random_F = RandomForestRegressor(n_estimators=n)
        random_F.fit(X_train, y_train)
        
        train_score = round(random_F.score(X_train, y_train), 3)
        test_score = round(random_F.score(X_test, y_test), 3)
        
        print(f'< {n}-estimators >')
        print('학습 데이터 세트 점수 :', train_score)
        print('테스트 데이터 세트 점수 :', test_score)
        print()

test_size = 0.1
< 10-estimators >
학습 데이터 세트 점수 : 0.998
테스트 데이터 세트 점수 : 0.993

test_size = 0.11
< 10-estimators >
학습 데이터 세트 점수 : 0.998
테스트 데이터 세트 점수 : 0.997

test_size = 0.12
< 10-estimators >
학습 데이터 세트 점수 : 0.998
테스트 데이터 세트 점수 : 0.992

test_size = 0.13
< 10-estimators >
학습 데이터 세트 점수 : 0.998
테스트 데이터 세트 점수 : 0.993

test_size = 0.14
< 10-estimators >
학습 데이터 세트 점수 : 0.998
테스트 데이터 세트 점수 : 0.994

test_size = 0.15
< 10-estimators >
학습 데이터 세트 점수 : 0.998
테스트 데이터 세트 점수 : 0.993

test_size = 0.16
< 10-estimators >
학습 데이터 세트 점수 : 0.998
테스트 데이터 세트 점수 : 0.988

test_size = 0.17
< 10-estimators >
학습 데이터 세트 점수 : 0.997
테스트 데이터 세트 점수 : 0.986

test_size = 0.18
< 10-estimators >
학습 데이터 세트 점수 : 0.998
테스트 데이터 세트 점수 : 0.991

test_size = 0.19
< 10-estimators >
학습 데이터 세트 점수 : 0.998
테스트 데이터 세트 점수 : 0.987

test_size = 0.2
< 10-estimators >
학습 데이터 세트 점수 : 0.997
테스트 데이터 세트 점수 : 0.983

test_size = 0.21
< 10-estimators >
학습 데이터 세트 점수 : 0.998
테스트 데이터 세트 점수 : 0.989

test_size = 0.22
< 10-estimators >
학습 데이터 세트 점수 : 0.99

```
test_size = 0.34
< 10-estimators >
학습 데이터 세트 점수 : 0.994
테스트 데이터 세트 점수 : 0.986
```

In [40]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

sel = ['총세대수', '지역', '임대건물구분', '공가수', '임대보증금', '임대료', '단지내주차면수']

X = train[sel]
y = train['등록차량수']
test_X = test[sel]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.34)

random_F = RandomForestRegressor(n_estimators=10)
random_F.fit(X_train, y_train)

pred = random_F.predict(test_X)
pred

array([680.5, 727.4, 727.4, ..., 226.7, 217.6, 182.7])

In [41]:
test['등록차량수'] = pred

test['코드별차량수평균'] = test.groupby('단지코드')['등록차량수'].transform(np.mean)

test

,level_0,index,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수,코드별차량수평균
0,0,0,C1072,5.727773,1.0,1,5,0.952031,1.233906,5.090909,7,0.449034,0.195940,0.0,2.0,7.507003,680.5,701.887500
1,1,1,C1072,5.727773,1.0,1,5,1.198010,0.311159,5.090909,0,0.731320,0.263924,0.0,2.0,7.507003,727.4,701.887500
2,2,2,C1072,5.727773,1.0,1,5,1.201163,1.190987,5.090909,7,0.731320,0.263924,0.0,2.0,7.507003,727.4,701.887500
3,3,3,C1072,5.727773,1.0,1,5,1.201163,1.276824,5.090909,7,0.731320,0.263924,0.0,2.0,7.507003,727.4,701.887500
4,4,4,C1072,5.727773,1.0,1,5,1.360945,0.633047,5.090909,7,0.890402,0.316928,0.0,2.0,7.507003,688.1,701.887500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,1003,1017,C1267,5.106216,1.0,2,9,0.846210,1.341202,13.818182,11,0.335075,0.143542,0.0,1.0,5.086835,406.2,401.754545
1003,1004,1018,C2189,2.800944,1.0,12,5,0.580609,1.019313,16.363636,7,0.108232,0.101540,0.0,2.0,3.215686,226.7,213.425000
1004,1005,1019,C2189,2.800944,1.0,12,5,0.580609,0.203863,16.363636,7,0.108232,0.101540,0.0,2.0,3.215686,226.7,213.425000
1005,1006,1020,C2189,2.800944,1.0,12,5,0.940117,2.156652,16.363636,7,0.247859,0.144758,0.0,2.0,3.215686,217.6,213.425000


In [44]:
test_new = test.drop_duplicates('단지코드', keep='first').reset_index()

sub_df = test_new[['단지코드', '코드별차량수평균']]
sub_df.columns = ['code', 'num']

sub_df

,code,num
0,C1072,701.887500
1,C1128,1002.500000
2,C1456,615.233333
3,C1840,495.450000
4,C1332,989.375000
...,...,...
142,C2456,293.875000
143,C1266,526.340000
144,C2152,52.700000
145,C1267,401.754545


In [1]:
sub_df.to_csv('07_19_first_submit.csv', index=False)

import os

os.listdir(os.getcwd())

NameError: name 'sub_df' is not defined